In [ ]:
import tensor_comprehensions as tc
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import logging
import time

In [ ]:
# as recommended by the authors for better performance
tune_settings = {
    "threads": 32,
    "generations": 8,
    "pop_size": 15,
    "number_elites": 2
}

In [ ]:
# heavier options
tune_settings = {
    "threads": 32,
    "generations": 20,
    "pop_size": 50,
    "number_elites": 3
}

In [ ]:
conv_lang = tc.database['convolution_strided']['lang']

In [ ]:
# try some different sizes

I = torch.randn(1, 3, 400, 200).cuda()
W1 = torch.randn(50, 3, 350, 190).cuda()
B = torch.randn(50).cuda()

In [ ]:
conv = tc.define(conv_lang.format(sh=1, sw=1), name='convolution_strided')

In [ ]:
conv.autotune(I, W1, B,
              cache='cache/big_conv.tc',
              **tune_settings,
              options=tc.Options('conv'))

In [ ]:
# run twice to prep cuda
out = conv(I, W1, B, cache='cache/tall_conv.tc')
torch.cuda.synchronize()
print(out.size())


In [ ]:
# test performance using TC
output = torch.zeros(1, 1, 51, 11).cuda()
torch.cuda.synchronize()

timings = np.zeros(10)

for i in range(10):
    start = time.perf_counter()
    output = conv(I, W1, B, cache='cache/tall_conv.tc', outputs=output)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)


In [ ]:
# test the torch equivalent

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 1, (350, 190)))
    
    def forward(self, x):
        return self.layer(x)

CN = ConvNet()
CN.cuda()

var_input = Variable(torch.randn(1, 3, 400, 200).cuda())
# run a couple times to work out start hiccups
CN(var_input)
torch.cuda.synchronize()
CN(var_input)
torch.cuda.synchronize()


# test performance
timings = np.zeros(10)

for i in range(10):
    start = time.perf_counter()
    output = CN(var_input)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)